## Contents

1. [Initial Data Exploration](https://www.kaggle.com/code/valentinbelyaev/1-0-intitial-eda-playground-series-s3e24)
2. [Building Baseline Models](https://www.kaggle.com/code/valentinbelyaev/2-0-baseline-model-playground-series-s3e24)
3. [Model ensembles & handle outliers](https://www.kaggle.com/valentinbelyaev/3-0-model-ensemble-playground-series-s3e24)
4. [Tunning XGBoost hyperparameters](https://www.kaggle.com/valentinbelyaev/4-0-model-ensemble-playground-series-s3e24)

*Notebooks №2 and №3 contained an incorrect metric for model evaluation (accuracy instead of ROC AUC)*

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e24/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s3e24/test.csv')

### Make sure everything is loaded correctly.

In [3]:
df_train.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


In [4]:
df_test.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,159256,40,165,70,84.0,1.2,1.2,1,1,130,...,186,49,115,14.2,1,0.9,19,25,32,0
1,159257,80,160,60,93.0,1.0,1.0,2,2,144,...,158,35,104,13.0,1,1.1,20,12,24,0
2,159258,60,170,70,86.5,0.6,0.7,1,1,117,...,173,39,88,15.4,1,1.4,38,60,36,0
3,159259,40,160,50,67.0,0.3,0.4,1,1,116,...,47,75,128,14.5,1,0.6,25,18,10,1
4,159260,40,170,75,89.4,1.0,0.9,1,1,132,...,100,39,123,16.5,1,1.0,30,39,27,1


In [5]:
print(f"Train data shape: {df_train.shape} | Test data shape: {df_test.shape}")

Train data shape: (159256, 24) | Test data shape: (106171, 23)


In [6]:
print(f"Number of NaN's in tranin data: {df_train.isna().sum().sum()}")
print(f"Number of NaN's in test data: {df_test.isna().sum().sum()}")

Number of NaN's in tranin data: 0
Number of NaN's in test data: 0


In [7]:
df_train.describe()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
count,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,...,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000
mean,79627.500000,44.306626,165.266929,67.143662,83.001990,1.005798,1.000989,1.023974,1.023421,122.503648,...,55.852684,114.607682,14.796965,1.074233,0.892764,25.516853,26.550296,36.216004,0.197996,0.437365
std,45973.391572,11.842286,8.818970,12.586198,8.957937,0.402113,0.392299,0.152969,0.151238,12.729315,...,13.964141,28.158931,1.431213,0.347856,0.179346,9.464882,17.753070,31.204643,0.398490,0.496063
min,0.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,...,9.000000,1.000000,4.900000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000,0.000000
25%,39813.750000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000,0.000000
50%,79627.500000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000,0.000000
75%,119441.250000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,32.000000,44.000000,0.000000,1.000000
max,159255.000000,85.000000,190.000000,130.000000,127.000000,9.900000,9.900000,2.000000,2.000000,213.000000,...,136.000000,1860.000000,21.000000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000,1.000000


In [8]:
df_test.describe()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
count,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,...,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000
mean,212341.000000,44.426538,165.221322,67.125618,82.999892,1.004776,0.999483,1.024216,1.023980,122.475403,...,127.406203,55.922286,114.597404,14.793354,1.072798,0.892463,25.559616,26.565427,36.086888,0.195458
std,30649.072052,11.897138,8.837065,12.586569,8.946584,0.397690,0.385752,0.153719,0.152988,12.765542,...,66.352488,14.000552,26.943401,1.434296,0.340818,0.179927,9.504442,17.944195,30.928138,0.396555
min,159256.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,71.000000,...,8.000000,18.000000,1.000000,5.000000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000
25%,185798.500000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,77.000000,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000
50%,212341.000000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,114.000000,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000
75%,238883.500000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,164.000000,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,33.000000,44.000000,0.000000
max,265426.000000,85.000000,190.000000,130.000000,127.700000,9.900000,9.900000,2.000000,2.000000,213.000000,...,548.000000,148.000000,1220.000000,21.100000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000


### Handle outliers in the data using IQR (Just for fun).

In [9]:
# Outliers columns were chosen by describe(). I don't think these features are 100% correct for outlier handling.
# If you have any suggestion on this step please let me know.
outlier_columns = ['LDL', 'AST', 'ALT', 'Gtp', 'fasting blood sugar', 'serum creatinine']

In [10]:
def handle_outliers(df):
    df_copy = df.copy()
    
    for col in outlier_columns:
        q1 = df_copy[col].quantile(0.25)
        q3 = df_copy[col].quantile(0.75)

        iqr = q3 - q1  # Interquartile range.

        df_copy[col] = df_copy[col].clip(lower=q1 - 1.5 * iqr, upper=q3 + 1.5 * iqr)  # Replace outliers with boundary values.

    return df_copy

In [11]:
df_train = handle_outliers(df_train)
df_test = handle_outliers(df_test)

### Check for outliers after replacing.

In [12]:
df_train.describe()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
count,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,...,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000
mean,79627.500000,44.306626,165.266929,67.143662,83.001990,1.005798,1.000989,1.023974,1.023421,122.503648,...,55.852684,114.489777,14.796965,1.074233,0.891970,25.102530,25.746999,33.731841,0.197996,0.437365
std,45973.391572,11.842286,8.818970,12.586198,8.957937,0.402113,0.392299,0.152969,0.151238,12.729315,...,13.964141,25.853617,1.431213,0.347856,0.172688,7.171154,12.465921,20.198613,0.398490,0.496063
min,0.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,...,9.000000,38.000000,4.900000,1.000000,0.500000,6.500000,1.000000,2.000000,0.000000,0.000000
25%,39813.750000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000,0.000000
50%,79627.500000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000,0.000000
75%,119441.250000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,32.000000,44.000000,0.000000,1.000000
max,159255.000000,85.000000,190.000000,130.000000,127.000000,9.900000,9.900000,2.000000,2.000000,213.000000,...,136.000000,190.000000,21.000000,6.000000,1.300000,42.500000,56.000000,83.000000,1.000000,1.000000


In [13]:
df_test.describe()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
count,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,...,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000
mean,212341.000000,44.426538,165.221322,67.125618,82.999892,1.004776,0.999483,1.024216,1.023980,122.475403,...,127.406203,55.922286,114.525256,14.793354,1.072798,0.891744,25.134086,25.866225,33.726206,0.195458
std,30649.072052,11.897138,8.837065,12.586569,8.946584,0.397690,0.385752,0.153719,0.152988,12.765542,...,66.352488,14.000552,25.866510,1.434296,0.340818,0.172598,7.185085,12.740216,20.136059,0.396555
min,159256.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,71.000000,...,8.000000,18.000000,38.000000,5.000000,1.000000,0.500000,6.500000,1.000000,2.000000,0.000000
25%,185798.500000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,77.000000,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000
50%,212341.000000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,114.000000,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000
75%,238883.500000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,164.000000,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,33.000000,44.000000,0.000000
max,265426.000000,85.000000,190.000000,130.000000,127.700000,9.900000,9.900000,2.000000,2.000000,213.000000,...,548.000000,148.000000,190.000000,21.100000,6.000000,1.300000,42.500000,58.500000,83.000000,1.000000


### Train & test split.

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X, y = df_train.drop(columns=['id', 'smoking'], axis=1), df_train['smoking']

In [16]:
X_test = df_test.drop(columns=['id'], axis=1)
test_ids = df_test['id']

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
print(f"Train shape: {X_train.shape}, {y_train.shape} | Validation shape: {X_valid.shape}, {y_valid.shape}")

Train shape: (111479, 22), (111479,) | Validation shape: (47777, 22), (47777,)


### Choose best baseline models from "Building Baseline Models" notebook.

### XGBoost

In [19]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [20]:
from sklearn.metrics import accuracy_score

print("XGBoost")

y_pred_train = xgb.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy score on training set: {accuracy_train*100}%")

y_pred_valid = xgb.predict(X_valid)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
print(f"Accuracy score on validation set: {accuracy_valid*100}%")

XGBoost
Accuracy score on training set: 82.15089837547879%
Accuracy score on validation set: 77.95173409799693%


### CatBoost

In [21]:
from catboost import CatBoostClassifier

cb = CatBoostClassifier(verbose=False)
cb.fit(X_train, y_train)

In [22]:
print("CatBoost")

y_pred_train = cb.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy score on training set: {accuracy_train*100}%")

y_pred_valid = cb.predict(X_valid)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
print(f"Accuracy score on validation set: {accuracy_valid*100}%")

CatBoost
Accuracy score on training set: 81.69610419899712%
Accuracy score on validation set: 78.27406492663835%


### LightGBM

In [23]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

LGBMClassifier()

In [24]:
print("LightGBM")

y_pred_train = lgbm.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy score on training set: {accuracy_train*100}%")

y_pred_valid = lgbm.predict(X_valid)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
print(f"Accuracy score on validation set: {accuracy_valid*100}%")

LightGBM
Accuracy score on training set: 79.22388970119933%
Accuracy score on validation set: 77.94545492601043%


### Create model ensembles.

### Hard voting.

In [25]:
from sklearn.ensemble import VotingClassifier

models = [
    ('XGBoost', XGBClassifier()),
    ('CatBoost', CatBoostClassifier(verbose=False)),
    ('LightGBM', LGBMClassifier())
]

ensemble = VotingClassifier(models, voting='hard')
ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('XGBoost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learnin...,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...)),
                             ('CatBoost',
                              <catboost.core.CatBoostClassifier object at 0x7ad56357e7a0>),
                             ('LightGBM', LGBMClassifier())])

In [26]:
print("Hard Voting")

y_pred_train = ensemble.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy score on training set: {accuracy_train*100}%")

y_pred_valid = ensemble.predict(X_valid)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
print(f"Accuracy score on validation set: {accuracy_valid*100}%")

Hard Voting
Accuracy score on training set: 81.17313574753989%
Accuracy score on validation set: 78.30964690122862%


In [27]:
y_pred_test = ensemble.predict(X_test)

predictions = pd.DataFrame({
    "id": test_ids,
    "smoking": y_pred_test
})

predictions.to_csv("hard_voting_baseline_models.csv", index=False)

### Soft voting.

In [28]:
from sklearn.ensemble import VotingClassifier

models = [
    ('XGBoost', XGBClassifier()),
    ('CatBoost', CatBoostClassifier(verbose=False)),
    ('LightGBM', LGBMClassifier())
]

ensemble = VotingClassifier(models, voting='soft')
ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('XGBoost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learnin...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...)),
                             ('CatBoost',
                              <catboost.core.CatBoostClassifier object at 0x7ad559811900>),
                             ('LightGBM', LGBMClassifier())],
                 voting='soft')

In [29]:
print("Soft Voting")

y_pred_train = ensemble.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy score on training set: {accuracy_train*100}%")

y_pred_valid = ensemble.predict(X_valid)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
print(f"Accuracy score on validation set: {accuracy_valid*100}%")

Soft Voting
Accuracy score on training set: 81.28795557907767%
Accuracy score on validation set: 78.2824371559537%


In [30]:
y_pred_test = ensemble.predict(X_test)

predictions = pd.DataFrame({
    "id": test_ids,
    "smoking": y_pred_test
})

predictions.to_csv("soft_voting_baseline_models.csv", index=False)

### Scores.
1. Hard voting - 0.7873
2. Soft voting - 0.7876